# Câu 1:

In [8]:
import sqlite3

# Kết nối đến cơ sở dữ liệu SQLite
conn = sqlite3.connect('demo.db')
cursor = conn.cursor()

# Tạo bảng Bang1 và thêm dữ liệu mẫu
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Bang1 (
        A REAL,
        B REAL
    );
''')
cursor.executemany('''
    INSERT INTO Bang1 (A, B)
    VALUES (?, ?);
''', [
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (5, 6)
])
conn.commit()

# Tính hệ số tương quan Pearson giữa A và B
cursor.execute('''
    SELECT 
        (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) / 
        SQRT(
            (COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) * 
            (COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))
        ) AS correlation_coefficient
    FROM Bang1;
''')
correlation_coefficient = cursor.fetchone()[0]

# In kết quả
print(f'Hệ số tương quan Pearson giữa A và B là: {correlation_coefficient}')

# Đóng kết nối
conn.close()

Hệ số tương quan Pearson giữa A và B là: 1.0


**Hệ số tương quan Pearson giữa A và B là 1.0, điều này cho thấy mối quan hệ tuyến tính hoàn hảo và tích cực giữa hai biến**

# Câu 2:

In [9]:
# import thư viện
import sqlite3
import numpy as np
import pandas as pd
import scipy.stats as stats

In [10]:
# Kết nối đến cơ sở dữ liệu SQLite
conn = sqlite3.connect('demo.db')
cursor = conn.cursor()

# Tạo bảng và thêm dữ liệu
cursor.execute('''
    CREATE TABLE IF NOT EXISTS car_scores (
        day TEXT,
        model TEXT,
        score REAL
    )
''')

data = [
    ('Day 1', 'A', 8), ('Day 1', 'B', 9), ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
    ('Day 3', 'A', 6), ('Day 3', 'B', 7), ('Day 3', 'C', 8),
    ('Day 4', 'A', 7), ('Day 4', 'B', 6), ('Day 4', 'C', 5)
]

cursor.executemany('INSERT INTO car_scores (day, model, score) VALUES (?, ?, ?)', data)
conn.commit()

# Đọc dữ liệu từ SQLite vào DataFrame
df = pd.read_sql_query('SELECT * FROM car_scores', conn)

# Kiểm tra và loại bỏ các hàng trùng lặp
df = df.drop_duplicates(subset=['day', 'model'])

# Chuyển đổi dữ liệu thành bảng pivot
pivot_table = df.pivot(index='day', columns='model', values='score')

# Thực hiện kiểm tra Chi-Square
chi2_stat, p_value, dof, expected = stats.chi2_contingency(pivot_table)

# Làm tròn bảng tần suất kỳ vọng
expected_rounded = np.round(expected, 2)

# In kết quả
print("Bảng dữ liệu:\n", pivot_table)
print("\nThống kê Chi-Square:", chi2_stat)
print("Giá trị P:", p_value)
print("Bậc tự do:", dof)
print("Tần suất kỳ vọng:\n", expected_rounded)

Bảng dữ liệu:
 model    A    B    C
day                 
Day 1  8.0  9.0  7.0
Day 2  7.5  8.5  7.0
Day 3  6.0  7.0  8.0
Day 4  7.0  6.0  5.0

Thống kê Chi-Square: 0.826439691310499
Giá trị P: 0.9913459315666546
Bậc tự do: 6
Tần suất kỳ vọng:
 [[7.95 8.51 7.53]
 [7.62 8.16 7.22]
 [6.96 7.45 6.59]
 [5.97 6.38 5.65]]


**Nhận xét:**
- Thống kê Chi-Square **(chi2_stat)**: Giá trị 0.8264 là rất nhỏ, cho thấy sự khác biệt giữa các giá trị quan sát và giá trị kỳ vọng là không đáng kể.

- Giá trị P **(p_value)**: Giá trị P là 0.9913, lớn hơn mức ý nghĩa thông thường (0.05). Điều này có nghĩa là không có đủ bằng chứng để bác bỏ giả thuyết không (H₀).

- Bậc tự do **(dof)**: Bậc tự do là 6, được tính dựa trên số hàng và cột của bảng dữ liệu.

- Tần suất kỳ vọng **(expected)**: Các giá trị kỳ vọng gần với giá trị quan sát, điều này củng cố thêm rằng không có sự khác biệt đáng kể.



# Câu 3:

In [12]:
cursor.execute('CREATE TABLE flights (id INTEGER PRIMARY KEY, departure_time INTEGER)')
data = [(1, 830), (2, 1445), (3, 1230), (4, 700), (5, 2200)]
cursor.executemany('INSERT INTO flights (id, departure_time) VALUES (?, ?)', data)
conn.commit()
cursor.execute('''
SELECT id, departure_time,
       printf('%02d:%02d %s', 
              departure_time / 100 % 12, 
              departure_time % 100, 
              CASE WHEN departure_time / 100 >= 12 THEN 'PM' ELSE 'AM' END
       ) AS formatted_time
FROM flights
''')
for row in cursor.fetchall():
    print(row)

(1, 830, '08:30 AM')
(2, 1445, '02:45 PM')
(3, 1230, '00:30 PM')
(4, 700, '07:00 AM')
(5, 2200, '10:00 PM')


# Câu 4:

In [16]:
import sqlite3
import numpy as np

# Kết nối đến cơ sở dữ liệu SQLite
conn = sqlite3.connect('demo.db')
cursor = conn.cursor()

# Tạo bảng và thêm dữ liệu mẫu
cursor.execute('''
    CREATE TABLE IF NOT EXISTS data_table (
        id INTEGER PRIMARY KEY,
        value REAL
    )
''')
data = [(1,), (2,), (3,), (4,), (5,), (100,)]  # Dữ liệu mẫu, bao gồm một giá trị ngoại lệ
cursor.executemany('INSERT INTO data_table (value) VALUES (?)', data)
conn.commit()

# Lấy dữ liệu từ bảng
cursor.execute('SELECT value FROM data_table')
values = [row[0] for row in cursor.fetchall()]

# Tính trung vị (median)
median = np.median(values)

# Tính độ lệch tuyệt đối (absolute deviation)
absolute_deviation = [abs(x - median) for x in values]

# Tính MAD (Median Absolute Deviation)
mad = np.median(absolute_deviation)

# Tìm các giá trị ngoại lệ (lớn hơn 1.5 lần MAD)
threshold = 1.5 * mad
outliers = list(set([x for x in values if abs(x - median) > threshold]))

# In kết quả
print(f"Trung vị (Median): {median}")
print(f"MAD: {mad}")
print(f"Threshold: {threshold}")
print(f"Các giá trị ngoại lệ: {outliers}")

# Đóng kết nối
conn.close()

Trung vị (Median): 3.5
MAD: 1.5
Threshold: 2.25
Các giá trị ngoại lệ: [1.0, 100.0]
